In [1]:
!pip install -r requirements.txt

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
import tensorflow as tf
import uuid
import numpy as np
import warnings
import rasterio
import numpy as np
import cv2
import json


from train.loader import Dataset


scale_factor = 0.25
loaded = {}

dataset = Dataset()
dataset.load_data()
dataset.scale_dataset(scale_factor)
dataset.split_train_test(test_size=0.25)

  0%|                                                                                                                               | 0/21377 [00:00<?, ?it/s]/opt/conda/lib/python3.12/site-packages/rasterio/__init__.py:355: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21377/21377 [00:56<00:00, 378.14it/s]


Number of training images: 16032
Number of validation images: 5345


In [4]:
import time
from tensorflow.keras.optimizers import Adam, AdamW, SGD, RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.optimizers.schedules import CosineDecay

from keras import backend as K

from train.experiment import Experiment

name = "experiment_13_tuning"
folder = f"experiments/{name}-{str(int(time.time()))}"

parameters = {
    "batch_size": 256,
    "epochs": 500
}

def build_model(hp):

    K.clear_session() # TF/Keras sucks

    model = Sequential()
    #model.add(Flatten(input_shape=(224, 224)))
    model.add(Flatten(input_shape=(int(224*scale_factor), int(224*scale_factor), 3))) # RGB
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float("initial_dropout", min_value=0.4, max_value=0.6, step=0.1)))

    layers_choices = []

    #layers = [16,8,4,2,1,0.5,0.25,0.125]

    first_layer_size = hp.Int("first_layer_size", min_value=1024, max_value=8*1024, step=2048)
    last_layer_size = hp.Int("last_layer_size", min_value=1024, max_value=8*1024, step=2048)

    layer_number = hp.Int("layer_count", min_value=2, max_value=5)

    layer_step = int((max(first_layer_size,last_layer_size)-min(first_layer_size,last_layer_size))/layer_number)

    
    for i in range(layer_number):

        #print(i, layer_number, first_layer_size, last_layer_size,layer_step, max(first_layer_size,last_layer_size)-min(first_layer_size,last_layer_size), int(first_layer_size+i*layer_step))

        if first_layer_size > last_layer_size:
            model.add(Dense(int(first_layer_size-(i*layer_step))))
        else:
            model.add(Dense(int(first_layer_size+(i*layer_step))))
            
        model.add(Activation('relu'))
        #model.add(Dropout(hp.Float(f"layer_{i}_dropout", min_value=0.0, max_value=1.0, step=0.05)))
        model.add(Dropout(0.15))
        model.add(BatchNormalization())
    
    model.add(Dense(12))
    model.add(Activation('softmax'))

    model.summary()
    
    base_lr = 0.01

    '''
    steps_epoch = 19239/256
    
    cosine_decay_scheduler = CosineDecay(
        0.01, 
        steps_epoch * 500, 
        alpha=0.0,
        name="CosineDecay",
        warmup_target=0.02,
        warmup_steps=steps_epoch * 25,
    )
    
    opt = AdamW(learning_rate=cosine_decay_scheduler, beta_1=0.9, beta_2=0.999, epsilon=1e-8, amsgrad=True, clipnorm=1.0)
    '''

    opt = AdamW(learning_rate=base_lr, beta_1=0.9, beta_2=0.999, epsilon=1e-8, amsgrad=True, clipnorm=1.0)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [5]:
experiment = Experiment(name, folder, parameters, build_model)

In [ ]:
from train.trainer import Trainer

trainer = Trainer(dataset, experiment)
#trainer.default_callbacks()
trainer.enable_lr_plateau(10)
trainer.enable_early_stop(50)
results = trainer.tuner_train(build_model, max_trials=10000)

Trial 1 Complete [00h 01m 22s]
val_accuracy: 0.6409728527069092

Best val_accuracy So Far: 0.6409728527069092
Total elapsed time: 00h 01m 22s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
0.4               |0.4               |initial_dropout
1024              |1024              |first_layer_size
1024              |1024              |last_layer_size
3                 |2                 |layer_count



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 9408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 9408)           │        37,632 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 9408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     9,634,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │        12,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 12)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,796,236 (45.00 MB)

 Trainable params: 11,771,276 (44.90 MB)

 Non-trainable params: 24,960 (97.50 KB)

Epoch 1/500
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 100ms/step - accuracy: 0.2812 - loss: 3.4076 - val_accuracy: 0.3620 - val_loss: 7.3984 - learning_rate: 0.0100
Epoch 2/500
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4348 - loss: 1.7633 - val_accuracy: 0.4490 - val_loss: 2.4285 - learning_rate: 0.0100
Epoch 3/500
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4614 - loss: 1.6729 - val_accuracy: 0.4992 - val_loss: 1.5812 - learning_rate: 0.0100
Epoch 4/500
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4941 - loss: 1.5428 - val_accuracy: 0.5287 - val_loss: 1.4836 - learning_rate: 0.0100
Epoch 5/500
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5216 - loss: 1.4884 - val_accuracy: 0.5038 - val_loss: 1.4984 - learning_rate: 0.0100
Epoch 6/500
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5303 - loss: 1.4482 - val_accuracy: 0.5542 - val_loss: 1.3466 - learning_rate: 0.0100
Epoch 7/500
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5634 - loss: 1.2836 - va

In [ ]:
#trainer.validate()

In [ ]:
#trainer.test()